In [ ]:
# libraries
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import messagebox

from PIL import ImageTk, Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

from keras.utils.np_utils import to_categorical 
from keras.models import load_model



In [ ]:
df = pd.read_csv("data\HAM10000_metadata.csv")
df

In [ ]:
df.info()

#### Note:
Dataframein(**df**) içerisinde **dx** feature(column)'unu countplot yapılıyor. Yani içerisinde bulunan kanser hücresinden kaç tane olduğunu ve dağılımını grafikle bize gösteriyor.
- **%matplotlib qt** eklentisinin düzgün çalışması(değerlerin doğru görüntülenebilmesi) için figure'un tam ekran yapılması gerekmektedir!

In [ ]:
%matplotlib qt5 
# %matplotlib widget # embedded
sns.countplot(x = "dx", data = df)

## 1-Data Preprocessing

- Öncelikle image'lerin dosya konumları(path) dataFrame'e path adıyla sütun olarak eklenir.

In [ ]:
img_folder_name = 'img/'
extension = ".jpg"
#sample image path = img_folder_name + image_id[i] + extension
# ismi path olan, dosya yoluna eşit değerleri olan yeni column oluşturulur.
df['path'] = [ img_folder_name + i + extension for i in df['image_id'] ]
df


- df['path']'in içerisindeki her bir satırı fonksiyon yapması için map kullanılır. 
- x = 'path' e karşılık gelir. 
- x pathindeki resmi aç diyoruz. 
- Ancak bu resmi biz array olarak tanımlamak ve tutmakistiyoruz, bu yüzden np.asarray cast'ı ekleriz.

In [ ]:
df["image"] = df["path"].map( lambda x: np.asarray(Image.open(x).resize((100,75))))

In [ ]:
df.head(5)

In [ ]:
%matplotlib inline
plt.imshow(df["image"][0]) # imshow: bir matrisi image şeklinde gösterir.

In [ ]:
# string değerleri numeric değerlere kodlar(convert).
df["dx_idx"] = pd.Categorical(df["dx"]).codes

In [ ]:
df.head(3)

In [ ]:
df.to_pickle("df.pkl") # verimizi depolayan bir dosya olduğunu bilmek yeterli.